In [1]:
%pip install pyLDAvis


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from gensim.parsing.preprocessing import stem_text
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim import corpora, models 
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
sys.path.append('../')
from utils import word_frequency_per_player, remove_similar_rows_per_player
from transformers import BertTokenizer, BertModel
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
import pyLDAvis
from pyLDAvis.gensim_models import prepare
import re




# Load data and convert to stemmed and BERT data 

In [3]:
# load data and drop NA's
df = pd.read_csv('https://github.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/raw/main/Preprocessing/data_clean/es_clean_2.csv')
df = df.dropna(subset=['data'])
df = remove_similar_rows_per_player(df, df['player'])

In [4]:
# create two copies 
df_stem = df.copy()

# create stemmed data
df_stem['data'] = df_stem['data'].apply(stem_text)

df_stem

,data,player,language,publishedAt
0,mercado fichaj fútbol directo con florian wirt...,Exequiel Palacios,es,2023-01-29T18:25:03Z
1,la última semana marzo primera ventana partido...,Exequiel Palacios,es,2023-01-30T16:52:46Z
2,gustavo puerta no jugará bayer leverkusen baye...,Exequiel Palacios,es,2023-01-31T20:41:38Z
3,alberto fernández el president afa claudio chi...,Exequiel Palacios,es,2023-02-09T18:32:38Z
4,alberto fernández también participaron campeon...,Exequiel Palacios,es,2023-02-12T21:13:55Z
...,...,...,...,...
68,ant minuto visitant llegado arco romano genera...,Piero Hincapié,es,2023-05-11T21:13:48Z
69,el bayer leverkusen jugó visita roma partido i...,Piero Hincapié,es,2023-05-11T20:56:21Z
70,un ecuatoriano volverá semifinal torneo uefa a...,Piero Hincapié,es,2023-05-11T18:30:25Z
71,la selección ecuador jugará do último amistoso...,Piero Hincapié,es,2023-05-10T23:37:55Z


In [5]:
tfidf = TfidfVectorizer(max_df=0.95, min_df=0.02)
df_tfidf = tfidf.fit_transform(df_stem.data).toarray()

In [6]:
df_tfidf

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.07328267, ..., 0.        , 0.11972684,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.06803261, 0.        , ..., 0.        , 0.06332175,
        0.10641498],
       [0.        , 0.        , 0.        , ..., 0.12317048, 0.        ,
        0.08653589],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [7]:
#Store the frequency matrix in data_tfidf. 
data_tfidf = pd.DataFrame(df_tfidf, columns=tfidf.vocabulary_.keys())

#data_tfidf['target_names']=sorted_df_ng.target_names.values #we can't use that because otherwise knn wouldn't work
data_tfidf.head()

,mercado,fichaj,fútbol,directo,con,florian,wirtz,lesión,exequiel,palacio,...,valladolid,genk,royal,antwerp,convocatoria,finalizó,tri,sornoza,melbourn,seleccionado
0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.073283,0.0,0.108667,0.0,0.0,0.091906,0.0,0.138405,...,0.0,0.0,0.075967,0.141842,0.0,0.0,0.0,0.0,0.119727,0.0
2,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.118080,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.206547,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0


In [8]:
#Store the feature names in a words list.
words = data_tfidf.columns.tolist()[:-2] 
words

['mercado',
 'fichaj',
 'fútbol',
 'directo',
 'con',
 'florian',
 'wirtz',
 'lesión',
 'exequiel',
 'palacio',
 'juego',
 'bayer',
 'leverkusen',
 'llevó',
 'inicio',
 'bien',
 'equipo',
 'bueno',
 'xabi',
 'alonso',
 'ni',
 'compañero',
 'dortmund',
 'la',
 'última',
 'semana',
 'marzo',
 'primera',
 'ventana',
 'partido',
 'internacional',
 'año',
 'asociación',
 'argentino',
 'afa',
 'combinado',
 'nacion',
 'disputará',
 'do',
 'amistoso',
 'estadio',
 'monument',
 'river',
 'plate',
 'ciudad',
 'air',
 'meno',
 'jugador',
 'director',
 'técnico',
 'ambo',
 'encuentro',
 'sin',
 'todavía',
 'objetivo',
 'tercer',
 'mundial',
 'historia',
 'probabl',
 'futbolista',
 'plantel',
 'qatar',
 'llamado',
 'ademá',
 'messi',
 'ángel',
 'di',
 'maría',
 'máximo',
 'referent',
 'arquero',
 'emiliano',
 'dibu',
 'martínez',
 'gerónimo',
 'rulli',
 'franco',
 'armani',
 'defensor',
 'nahuel',
 'molina',
 'gonzalo',
 'montiel',
 'germán',
 'pezzella',
 'nicolá',
 'otamendi',
 'cristian',
 'cut

In [9]:
kmeans = KMeans(n_clusters = 4, max_iter=1000, random_state=42)
kmeans.fit(data_tfidf)

KMeans(max_iter=1000, n_clusters=4, random_state=42)

In [10]:
common_words = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]

In [11]:
for num, centroid in enumerate(common_words):
    words_in_centroid = [words[word] for word in centroid if word < len(words)]
    print(f"{num}: {', '.join(words_in_centroid)}")
# By adding the if word < len(words) condition, we ensure that only valid indices are used to access the words list or dictionary.

0: sevilla, tiro, contrato, pelota, embargo, ángel, roma, demanda, segundo, ganar
1: bundesliga, hradecki, vivo, luca, periodista, romero, tiro, sevilla, todavía, juventu
2: banfield, ofici, panamá, antwerp, conscient, mostró, seguir, bayarena, fútbol, penalti
3: amarilla, octavo, contrato, pelota, gole, compromiso, ángel, di, liga, alemania


In [12]:
df_stem['cluster'] = kmeans.labels_
df_stem

,data,player,language,publishedAt,cluster
0,mercado fichaj fútbol directo con florian wirt...,Exequiel Palacios,es,2023-01-29T18:25:03Z,3
1,la última semana marzo primera ventana partido...,Exequiel Palacios,es,2023-01-30T16:52:46Z,2
2,gustavo puerta no jugará bayer leverkusen baye...,Exequiel Palacios,es,2023-01-31T20:41:38Z,3
3,alberto fernández el president afa claudio chi...,Exequiel Palacios,es,2023-02-09T18:32:38Z,3
4,alberto fernández también participaron campeon...,Exequiel Palacios,es,2023-02-12T21:13:55Z,3
...,...,...,...,...,...
68,ant minuto visitant llegado arco romano genera...,Piero Hincapié,es,2023-05-11T21:13:48Z,3
69,el bayer leverkusen jugó visita roma partido i...,Piero Hincapié,es,2023-05-11T20:56:21Z,0
70,un ecuatoriano volverá semifinal torneo uefa a...,Piero Hincapié,es,2023-05-11T18:30:25Z,0
71,la selección ecuador jugará do último amistoso...,Piero Hincapié,es,2023-05-10T23:37:55Z,1


In [13]:
clusters = df_stem.groupby(['cluster']).size()
clusters

cluster
0    3708
1    1981
2    3783
3    7570
dtype: int64

# LDA Model

In [14]:
# Split the texts into a list of words
texts = [text.split() for text in df_stem['data']]

# Create the Gensim dictionary
dictionary = corpora.Dictionary(texts)

# Remove rare and common words from the dictionary
dictionary.filter_extremes(no_below=118, no_above=0.95)

# Derive the absolute frequency matrix using doc2bow
corpus = [dictionary.doc2bow(text) for text in texts]


In [15]:
lda = models.LdaModel(corpus, num_topics=4, id2word=dictionary, chunksize=10, iterations=100, passes=10, random_state=42)


In [16]:
lda.show_topics()

[(0,
  '0.025*"félix" + 0.022*"ecuador" + 0.021*"piero" + 0.021*"moisé" + 0.020*"hincapié" + 0.018*"sánchez" + 0.018*"selección" + 0.014*"caicedo" + 0.012*"australia" + 0.011*"la"'),
 (1,
  '0.031*"piero" + 0.022*"hincapié" + 0.019*"año" + 0.015*"no" + 0.015*"jugador" + 0.014*"do" + 0.013*"europa" + 0.012*"la" + 0.012*"ecuatoriano" + 0.011*"gran"'),
 (2,
  '0.035*"hincapié" + 0.034*"el" + 0.030*"piero" + 0.023*"leverkusen" + 0.019*"ecuatoriano" + 0.018*"bayer" + 0.011*"minuto" + 0.011*"no" + 0.009*"partido" + 0.009*"leagu"'),
 (3,
  '0.031*"europa" + 0.027*"leagu" + 0.026*"el" + 0.025*"leverkusen" + 0.022*"bayer" + 0.022*"en" + 0.019*"roma" + 0.018*"partido" + 0.016*"final" + 0.013*"equipo"')]

In [17]:
topics=lda[corpus]
df_stem['lda_predicted_clusters']= [max(topics[i],key=lambda item:item[1])[0] for i in range(len(topics))]

In [18]:
df_stem.head()

,data,player,language,publishedAt,cluster,lda_predicted_clusters
0,mercado fichaj fútbol directo con florian wirt...,Exequiel Palacios,es,2023-01-29T18:25:03Z,3,2
1,la última semana marzo primera ventana partido...,Exequiel Palacios,es,2023-01-30T16:52:46Z,2,1
2,gustavo puerta no jugará bayer leverkusen baye...,Exequiel Palacios,es,2023-01-31T20:41:38Z,3,1
3,alberto fernández el president afa claudio chi...,Exequiel Palacios,es,2023-02-09T18:32:38Z,3,0
4,alberto fernández también participaron campeon...,Exequiel Palacios,es,2023-02-12T21:13:55Z,3,1


In [19]:
pyLDAvis.enable_notebook()
LDAvis_prepared = prepare(lda, corpus, dictionary)
LDAvis_prepared



PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.079627  0.002922       1        1  36.652559
1     -0.120482 -0.278644       2        1  24.286362
0     -0.231760  0.216050       3        1  20.313805
3      0.272615  0.059673       4        1  18.747274, topic_info=          Term          Freq         Total Category  logprob  loglift
1660     félix   6931.000000   6931.000000  Default  30.0000  30.0000
308     europa  15007.000000  15007.000000  Default  29.0000  29.0000
138         el  25987.000000  25987.000000  Default  28.0000  28.0000
1694   ecuador   7621.000000   7621.000000  Default  27.0000  27.0000
234      leagu  13273.000000  13273.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
699       uefa   2380.812177   5997.226577   Topic4  -4.6794   0.7503
667   escuadra   1590.243115   2551.771714   Topic4  -5.0829   1.2012
7          con   1515.117838   3246.577910   Topic4  -5.1313   0.9120
156         no   1659.296567  13113.838678   Topic4  -5.0404  -0.3932
87       marzo   1439.030301   4017.518953   Topic4  -5.1828   0.6474

[281 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
287       1  0.993989          a
287       2  0.000299          a
287       4  0.005683          a
1717      2  0.007529  actuacion
1717      4  0.992611  actuacion
...     ...       ...        ...
1747      2  0.000479     ángelo
1747      3  0.999376     ángelo
428       1  0.999284       área
428       2  0.000438       área
1035      2  0.999098      único

[516 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1, 4])

## The names of the players take a huge influence on the prediciton therefore we remove them in the next step

In [20]:
df_stem['data']

0     mercado fichaj fútbol directo con florian wirt...
1     la última semana marzo primera ventana partido...
2     gustavo puerta no jugará bayer leverkusen baye...
3     alberto fernández el president afa claudio chi...
4     alberto fernández también participaron campeon...
                            ...                        
68    ant minuto visitant llegado arco romano genera...
69    el bayer leverkusen jugó visita roma partido i...
70    un ecuatoriano volverá semifinal torneo uefa a...
71    la selección ecuador jugará do último amistoso...
72    auspiciado uno gancho ecuatoriano semifinal eu...
Name: data, Length: 17042, dtype: object

In [21]:
# Function to remove specific words from the string
def remove_words(text):
    pattern = r"\b(mitchel|bakker|exequiel|palacios|piero|hincapie|jeremie|frimpong|jonathan|tah|moussa|diaby|mykhaylo|mudryk)\b"
    return re.sub(pattern, "", text)

# Apply the function to the data column
df_stem['data'] = df_stem['data'].apply(lambda x: remove_words(str(x)))

df_stem


,data,player,language,publishedAt,cluster,lda_predicted_clusters
0,mercado fichaj fútbol directo con florian wirt...,Exequiel Palacios,es,2023-01-29T18:25:03Z,3,2
1,la última semana marzo primera ventana partido...,Exequiel Palacios,es,2023-01-30T16:52:46Z,2,1
2,gustavo puerta no jugará bayer leverkusen baye...,Exequiel Palacios,es,2023-01-31T20:41:38Z,3,1
3,alberto fernández el president afa claudio chi...,Exequiel Palacios,es,2023-02-09T18:32:38Z,3,0
4,alberto fernández también participaron campeon...,Exequiel Palacios,es,2023-02-12T21:13:55Z,3,1
...,...,...,...,...,...,...
68,ant minuto visitant llegado arco romano genera...,Piero Hincapié,es,2023-05-11T21:13:48Z,3,2
69,el bayer leverkusen jugó visita roma partido i...,Piero Hincapié,es,2023-05-11T20:56:21Z,0,2
70,un ecuatoriano volverá semifinal torneo uefa a...,Piero Hincapié,es,2023-05-11T18:30:25Z,0,3
71,la selección ecuador jugará do último amistoso...,Piero Hincapié,es,2023-05-10T23:37:55Z,1,0


In [22]:
lda_2 = models.LdaModel(corpus, num_topics=10, id2word=dictionary, chunksize=10, iterations=100, passes=10, random_state=42)


## Finding the best number of topics

In [23]:
models_arr = {'iteration_' + str(iter): '' for iter in range(2, 3)}
for iter in range(2, 15): 
    models_arr.update({'iteration_'+ str(iter): models.LdaModel(corpus, num_topics=iter, id2word=dictionary, chunksize=10, iterations=100, passes=10, random_state=42)})


In [24]:
models_arr['iteration_5'].show_topics()

[(0,
  '0.030*"félix" + 0.027*"ecuador" + 0.026*"moisé" + 0.022*"sánchez" + 0.021*"selección" + 0.019*"piero" + 0.019*"caicedo" + 0.015*"estupiñán" + 0.015*"pervi" + 0.014*"australia"'),
 (1,
  '0.031*"año" + 0.030*"club" + 0.024*"do" + 0.020*"jugador" + 0.020*"piero" + 0.017*"vall" + 0.014*"mejorar" + 0.014*"independient" + 0.013*"que" + 0.013*"°"'),
 (2,
  '0.020*"no" + 0.017*"leagu" + 0.014*"la" + 0.013*"desd" + 0.012*"esmeralda" + 0.011*"futbolista" + 0.011*"momento" + 0.010*"europa" + 0.010*"al" + 0.009*"año"'),
 (3,
  '0.052*"europa" + 0.049*"leagu" + 0.029*"bayer" + 0.029*"leverkusen" + 0.025*"en" + 0.024*"roma" + 0.023*"el" + 0.022*"final" + 0.021*"partido" + 0.014*"ida"'),
 (4,
  '0.050*"hincapié" + 0.042*"el" + 0.037*"piero" + 0.027*"leverkusen" + 0.025*"ecuatoriano" + 0.020*"bayer" + 0.012*"uefa" + 0.012*"minuto" + 0.012*"no" + 0.011*"en"')]

In [25]:
pyLDAvis.enable_notebook()
LDAvis_prepared = prepare(models_arr['iteration_5'], corpus, dictionary)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.019415 -0.110422       1        1  38.147014
0     -0.337645  0.012003       2        1  18.448036
1     -0.019431  0.276570       3        1  17.186818
3      0.065569 -0.290738       4        1  14.987747
2      0.272091  0.112587       5        1  11.230385, topic_info=            Term          Freq         Total Category  logprob  loglift
374     hincapié  26110.000000  26110.000000  Default  30.0000  30.0000
308       europa  12262.000000  12262.000000  Default  29.0000  29.0000
234        leagu  12766.000000  12766.000000  Default  28.0000  28.0000
20    leverkusen  19806.000000  19806.000000  Default  27.0000  27.0000
2          bayer  16401.000000  16401.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
126       volant   1105.112975   2992.558015   Topic5  -4.9344   1.1904
1580        mayo   1036.090147   4020.392227   Topic5  -4.9989   0.8306
59            do   1095.528980   9744.841079   Topic5  -4.9431   0.0010
504      español    957.170754   2337.577128   Topic5  -5.0781   1.2937
241         part    957.674159   3582.594070   Topic5  -5.0776   0.8672

[294 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
287       1  0.816902         a
287       3  0.000268         a
287       4  0.003488         a
287       5  0.179477         a
875       3  0.999636  absoluta
...     ...       ...       ...
471       4  0.023216         °
1747      2  0.999643    ángelo
1747      3  0.000443    ángelo
428       1  0.999462      área
428       3  0.000402      área

[551 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 2, 4, 3])

In [26]:
pyLDAvis.enable_notebook()
LDAvis_prepared = prepare(models_arr['iteration_7'], corpus, dictionary)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.185045  0.011942       1        1  23.440676
5     -0.106653 -0.115411       2        1  17.331876
1     -0.013834 -0.060616       3        1  14.773129
0      0.261539  0.164417       4        1  14.606447
4     -0.231498 -0.087090       5        1  12.038552
2      0.282371 -0.254117       6        1   8.911580
6     -0.006879  0.340875       7        1   8.897740, topic_info=          Term          Freq         Total Category  logprob  loglift
374   hincapié  23935.000000  23935.000000  Default  30.0000  30.0000
377      piero  22817.000000  22817.000000  Default  29.0000  29.0000
103    partido  13537.000000  13537.000000  Default  28.0000  28.0000
1660     félix  11321.000000  11321.000000  Default  27.0000  27.0000
1694   ecuador  11514.000000  11514.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
1692   caicedo   2954.625296   7872.817575   Topic7  -3.7182   1.4393
306         en   3075.532650  14417.804238   Topic7  -3.6781   0.8744
380   alemania   1665.289101   3550.353755   Topic7  -4.2915   1.6623
242    premier   1253.150852   1719.121789   Topic7  -4.5759   2.1032
234      leagu   1796.105647  12811.617530   Topic7  -4.2159   0.4546

[368 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
287       1  0.246274       a
287       3  0.753733       a
586       5  0.998763   abril
288       2  0.187400  acción
288       7  0.812269  acción
...     ...       ...     ...
1737      1  0.425145   énner
1737      6  0.574365   énner
129       3  0.999353  última
350       3  0.944221  último
350       7  0.055201  último

[471 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 2, 1, 5, 3, 7])